# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb9923d2c40>
├── 'a' --> tensor([[ 0.9089,  0.1325, -1.1437],
│                   [ 1.2786, -0.5077,  0.3543]])
└── 'x' --> <FastTreeValue 0x7fb9923d2bb0>
    └── 'c' --> tensor([[-1.1352, -0.5879,  0.0642,  0.3671],
                        [ 0.0254, -0.7781, -1.0237, -1.2401],
                        [-0.3684,  0.3264,  1.2216,  0.4826]])

In [4]:
t.a

tensor([[ 0.9089,  0.1325, -1.1437],
        [ 1.2786, -0.5077,  0.3543]])

In [5]:
%timeit t.a

84.4 ns ± 0.158 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb9923d2c40>
├── 'a' --> tensor([[ 0.6555, -0.0871,  0.5035],
│                   [-0.8578, -0.6109,  0.3628]])
└── 'x' --> <FastTreeValue 0x7fb9923d2bb0>
    └── 'c' --> tensor([[-1.1352, -0.5879,  0.0642,  0.3671],
                        [ 0.0254, -0.7781, -1.0237, -1.2401],
                        [-0.3684,  0.3264,  1.2216,  0.4826]])

In [7]:
%timeit t.a = new_value

89.4 ns ± 0.162 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9089,  0.1325, -1.1437],
               [ 1.2786, -0.5077,  0.3543]]),
    x: Batch(
           c: tensor([[-1.1352, -0.5879,  0.0642,  0.3671],
                      [ 0.0254, -0.7781, -1.0237, -1.2401],
                      [-0.3684,  0.3264,  1.2216,  0.4826]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9089,  0.1325, -1.1437],
        [ 1.2786, -0.5077,  0.3543]])

In [11]:
%timeit b.a

65 ns ± 0.133 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6958,  0.0032, -1.0416],
               [-0.1704, -0.9345,  0.7123]]),
    x: Batch(
           c: tensor([[-1.1352, -0.5879,  0.0642,  0.3671],
                      [ 0.0254, -0.7781, -1.0237, -1.2401],
                      [-0.3684,  0.3264,  1.2216,  0.4826]]),
       ),
)

In [13]:
%timeit b.a = new_value

551 ns ± 11.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

915 ns ± 0.207 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 29.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

142 µs ± 652 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb8c24dc8b0>
├── 'a' --> tensor([[[ 0.9089,  0.1325, -1.1437],
│                    [ 1.2786, -0.5077,  0.3543]],
│           
│                   [[ 0.9089,  0.1325, -1.1437],
│                    [ 1.2786, -0.5077,  0.3543]],
│           
│                   [[ 0.9089,  0.1325, -1.1437],
│                    [ 1.2786, -0.5077,  0.3543]],
│           
│                   [[ 0.9089,  0.1325, -1.1437],
│                    [ 1.2786, -0.5077,  0.3543]],
│           
│                   [[ 0.9089,  0.1325, -1.1437],
│                    [ 1.2786, -0.5077,  0.3543]],
│           
│                   [[ 0.9089,  0.1325, -1.1437],
│                    [ 1.2786, -0.5077,  0.3543]],
│           
│                   [[ 0.9089,  0.1325, -1.1437],
│                    [ 1.2786, -0.5077,  0.3543]],
│           
│                   [[ 0.9089,  0.1325, -1.1437],
│                    [ 1.2786, -0.5077,  0.3543]]])
└── 'x' --> <FastTreeValue 0x7fb8c24dcd30>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.6 µs ± 60.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb8ca618f70>
├── 'a' --> tensor([[ 0.9089,  0.1325, -1.1437],
│                   [ 1.2786, -0.5077,  0.3543],
│                   [ 0.9089,  0.1325, -1.1437],
│                   [ 1.2786, -0.5077,  0.3543],
│                   [ 0.9089,  0.1325, -1.1437],
│                   [ 1.2786, -0.5077,  0.3543],
│                   [ 0.9089,  0.1325, -1.1437],
│                   [ 1.2786, -0.5077,  0.3543],
│                   [ 0.9089,  0.1325, -1.1437],
│                   [ 1.2786, -0.5077,  0.3543],
│                   [ 0.9089,  0.1325, -1.1437],
│                   [ 1.2786, -0.5077,  0.3543],
│                   [ 0.9089,  0.1325, -1.1437],
│                   [ 1.2786, -0.5077,  0.3543],
│                   [ 0.9089,  0.1325, -1.1437],
│                   [ 1.2786, -0.5077,  0.3543]])
└── 'x' --> <FastTreeValue 0x7fb8c24dc8e0>
    └── 'c' --> tensor([[-1.1352, -0.5879,  0.0642,  0.3671],
                        [ 0.0254, -0.7781, -1.0237, -1.2401],
                 

In [23]:
%timeit t_cat(trees)

31.4 µs ± 44.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.9 µs ± 98.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.9089,  0.1325, -1.1437],
                [ 1.2786, -0.5077,  0.3543]],
       
               [[ 0.9089,  0.1325, -1.1437],
                [ 1.2786, -0.5077,  0.3543]],
       
               [[ 0.9089,  0.1325, -1.1437],
                [ 1.2786, -0.5077,  0.3543]],
       
               [[ 0.9089,  0.1325, -1.1437],
                [ 1.2786, -0.5077,  0.3543]],
       
               [[ 0.9089,  0.1325, -1.1437],
                [ 1.2786, -0.5077,  0.3543]],
       
               [[ 0.9089,  0.1325, -1.1437],
                [ 1.2786, -0.5077,  0.3543]],
       
               [[ 0.9089,  0.1325, -1.1437],
                [ 1.2786, -0.5077,  0.3543]],
       
               [[ 0.9089,  0.1325, -1.1437],
                [ 1.2786, -0.5077,  0.3543]]]),
    x: Batch(
           c: tensor([[[-1.1352, -0.5879,  0.0642,  0.3671],
                       [ 0.0254, -0.7781, -1.0237, -1.2401],
                       [-0.3684,  0.3264,  1.2216,  0.4826]],
         

In [26]:
%timeit Batch.stack(batches)

84.6 µs ± 186 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.9089,  0.1325, -1.1437],
               [ 1.2786, -0.5077,  0.3543],
               [ 0.9089,  0.1325, -1.1437],
               [ 1.2786, -0.5077,  0.3543],
               [ 0.9089,  0.1325, -1.1437],
               [ 1.2786, -0.5077,  0.3543],
               [ 0.9089,  0.1325, -1.1437],
               [ 1.2786, -0.5077,  0.3543],
               [ 0.9089,  0.1325, -1.1437],
               [ 1.2786, -0.5077,  0.3543],
               [ 0.9089,  0.1325, -1.1437],
               [ 1.2786, -0.5077,  0.3543],
               [ 0.9089,  0.1325, -1.1437],
               [ 1.2786, -0.5077,  0.3543],
               [ 0.9089,  0.1325, -1.1437],
               [ 1.2786, -0.5077,  0.3543]]),
    x: Batch(
           c: tensor([[-1.1352, -0.5879,  0.0642,  0.3671],
                      [ 0.0254, -0.7781, -1.0237, -1.2401],
                      [-0.3684,  0.3264,  1.2216,  0.4826],
                      [-1.1352, -0.5879,  0.0642,  0.3671],
                      [ 0.0254, -

In [28]:
%timeit Batch.cat(batches)

155 µs ± 682 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

351 µs ± 3.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
